In [ ]:
import paho.mqtt.client as mqtt
import mysql.connector
import time

In [ ]:
def on_connect(client, userdata, flags, rc):
  print("Connected with result code "+str(rc))
  client.subscribe("dataTetas")

def on_message(client, userdata, msg):
  if(msg.topic == 'dataTetas'):
    print(msg.topic + " " + str(msg.payload))
    coba = msg.payload
    print("msg Payload : " , str(coba))
    coba2 = coba.decode()
    print(coba2)

    test = coba2.split(",")
    temp = test[0]
    hum = test[1]
    stat = test[2]

    con = mysql.connector.connect(
        host = "localhost",
        user = "root",
        database = "inkubator"
    )

    cur = con.cursor()

    cime = time.strftime("%y:%m:%d")
    ime = time.strftime("%H:%M:%S")

    value = (cime, ime, temp, hum, stat)
    query = "INSERT INTO rekap (tanggal, waktu, suhu, kelembpan, status) VALUES (%s,%s, %s, %s, %s)"
    cur.executemany(query,(value,))
    time.sleep(0.5)
    con.commit()
    cur.close()
    con.close()

In [ ]:
client = mqtt.Client()
client.loop_start()
client.subscribe("dataTetas")
client.on_connect = on_connect
client.on_message = on_message
client.connect("broker.mqtt-dashboard.com", 1883)
time.sleep(60)
client.loop_stop()